In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

# Turn interactive plotting off
plt.ioff()

def calc_coords_in_circle(x, y, radius, x_size, y_size):
    """ Calculate coordinates in circle that satisfy radius & grid (must be located within) conditions
    
    :param x:      Current x coordinate in a grid
    :param y:      Current y coordinate in a grid
    :param radius: Desired radius of a circle
    :param x_size: Grid x size (of map to place people into)
    :param y_size: Grid y size (of map to place people into)
    :return:       Satisfying coordinates
    """
    
    coords = []

    # Possible y-shifts
    for i in range(-int(np.floor(radius)), int(np.floor(radius)) + 1):
        
        # Possible x-shifts
        for j in range(int(np.floor(radius)) + 1):
            
            # Coords in circle
            if np.sqrt(i ** 2 + j ** 2) <= radius:
                
                # Append valid coordinates
                if (x + j) >= 0 and (y + i) >= 0 and (x + j) < x_size and (y + i) < y_size:
                    coords.append((x + j, y + i))
                
                # Append valid mirrored coordinates
                if j > 0:
                    
                    if (x - j) >= 0 and (y + i) >= 0 and (x - j) < x_size and (y + i) < y_size:
                        coords.append((x - j, y + i))
                
            # Coords not in circle
            else:
                
                break

    return np.array(coords)


def transport_iter(people_x_arr, people_y_arr, init_people_x_arr, init_people_y_arr, x_size, y_size, radius):
    """ Transport people from current locations based on radius & map (grid) conditions
    
    :param people_x_arr:      Array of x coordinates (current position) for all people
    :param people_y_arr:      Array of y coordinates (current position) for all people
    :param init_people_x_arr: Initial array of x coordinates for all people - needed to calc person's allowed neighbourhood
    :param init_people_y_arr: Initial array of y coordinates for all people - needed to calc person's allowed neighbourhood
    :param x_size:            Grid x size (of map to place people into)
    :param y_size:            Grid y size (of map to place people into)
    :param radius:            Limiting radius for one epoch transportation
    """

    for i, (x, y) in enumerate(zip(people_x_arr, people_y_arr)):

        valid_coords = False
        
        # Get possible moves from current (x, y) location
        coords = calc_coords_in_circle(x, y, radius, x_size, y_size)
        
        # Randomly choose new move that yields neighbourhood coordinates
        while not valid_coords:
            
            rnd_move_idx = np.random.choice(coords.shape[0], 1, replace=False)[0]
            new_x, new_y = coords[rnd_move_idx]
            if np.sqrt((new_x - init_people_x_arr[i]) ** 2 + (new_y - init_people_y_arr[i]) ** 2) <= neighbourhood_radius:
                valid_coords = True
                
        people_x_arr[i] = new_x
        people_y_arr[i] = new_y

        
def make_disease_matrix(people_x_arr, people_y_arr, people_observ_arr, x_size, y_size, spread_radius):
    """ Make disease (exposure) matrix which indicates for each spot a number of ill people near, that can transfer a disease
    
    :param people_x_arr:      Array of x coordinates (current position) for all people
    :param people_y_arr:      Array of y coordinates (current position) for all people
    :param people_observ_arr: People with observable disease that can transmit it
    :param x_size:            Grid x size (of map to place people into)
    :param y_size:            Grid y size (of map to place people into)
    :param spread_radius:     Radius of exposure of an infected person within which disease can be transfered to a healthy one
    :return:                  Disease (exposure) matrix
    """
    
    # Init disease matrix
    disease_mat = np.zeros(shape=(y_size, x_size)).astype(int)

    # Get indices of diseased (infected) people
    disease_indices = np.where(people_observ_arr == 1)[0]

    # Calculate disease spread for each infected person
    for i in disease_indices:

        # infected_coords = calc_infected_coords(people_x_arr[i], people_y_arr[i], spread_radius, x_size, y_size)
        infected_coords = calc_coords_in_circle(people_x_arr[i], people_y_arr[i], spread_radius, x_size, y_size)
        for x, y in infected_coords:
            disease_mat[y, x] += 1
            
    return disease_mat


def calc_infection_prob(infection_exposure, infect_prob = 0.5):
    """ Calculate infection probability based on exposure and infection's base probability
    
    :param infection_exposure: Number of infected people which can transfer the disease and located near a healthy person
    :param infect_prob:        Infection's base probability
    :return:                   Probability of infection
    """
    return 1 - (1 - infect_prob) ** infection_exposure


def spread_disease(disease_mat, people_x_arr, people_y_arr, people_status_arr, people_timer_arr, infect_prob):
    """ Spread disease between infected and healthy people based on disease_mat. Updates people_status_arr & people_timer_arr
    
    :param disease_mat:       Map of overlapping areas of disease exposure from current infected people
    :param people_x_arr:      Array of x coordinates (current position) for all people
    :param people_y_arr:      Array of y coordinates (current position) for all people
    :param people_status_arr: People's true status of having a disease - 1 = if has, 0 = if not
    :param people_timer_arr:  Array of timers (expressed in epochs until) for each person, which indicate when person can transfer disease (if person was infected)
    :param infect_prob:       Disease's base probability of transmission
    """
    
    for i, (x, y) in enumerate(zip(people_x_arr, people_y_arr)):

        # Get infection exposure for person
        infection_exposure = disease_mat[y, x]

        # Calculate probability of getting ill and illness' outcome
        infection_prob = calc_infection_prob(infection_exposure, infect_prob)
        infection_outcome = np.random.choice([0, 1], p=[1 - infection_prob, infection_prob], replace=False)

        # Infect only healthy people
        if infection_outcome == 1 and people_status_arr[i] == 0 and people_timer_arr[i] == 0:
            people_status_arr[i] = infection_outcome
            people_timer_arr[i] = timer_start

            
def plot_disease_exposure(people_x_arr, people_y_arr, people_observ_arr, people_status_arr, spread_radius, epoch=None, path=None):
    """ Plot disease (exposure) matrix
    
    :param people_x_arr:       Array of x coordinates (current position) for all people
    :param people_y_arr:       Array of y coordinates (current position) for all people
    :param people_observ_arr:  Array that indicates ill people with transfer disease status
    :param people_status_arr:  Array that indicates all ill people with both transfer (visible) and invisible disease status
    :param spread_radius:      Disease spreading radius
    :param epoch:              Current epoch in simulation
    :param path:               Absolute path to save plot to
    """
    
    fig, ax = plt.subplots(figsize=(15, 15))
    ax.scatter(people_x_arr, people_y_arr, s=1, c='grey', alpha=0.1, label='Uninfected')
    ax.scatter(people_x_arr[people_status_arr == 1], people_y_arr[people_status_arr == 1], s=spread_radius, c='yellow', alpha=0.5, label='Infected (visible + invisible)')
    ax.scatter(people_x_arr[people_observ_arr == 1], people_y_arr[people_observ_arr == 1], s=spread_radius, c='red', alpha=1, label='Infected (visible = transmitters)')
    ax.set_title(f'Disease (exposure) matrix - visibly ill people = {people_observ_arr.sum()}, infected total = {people_status_arr.sum()} - epoch {epoch}')
    ax.legend()

    if epoch is not None and path is not None:

        # Create folder if it doesn't exist
        if not os.path.exists(path):
            os.mkdir(path)

        # Save plot
        fig.savefig(os.path.join(path, f'Disease_matrix_epoch_{epoch}.png'), dpi=300)

    plt.close(fig)

            
def simulate_transportations_with_infections(people_num, x_size, y_size, infected_init_num, timer_start, neighbourhood_radius, infect_prob, radius, spread_radius, epochs, plot_disease_matrix=None):
    """ Simulate people transportation and disease spread in a square grid
    
    :param people_num:           Number of people in simulation
    :param x_size:               Grid x size (of map to place people into)
    :param y_size:               Grid y size (of map to place people into)
    :param infected_init_num:    Initial infected people number
    :param timer_start:          Steps (epochs) until infected person can transmit a disease (exception: initial group)
    :param neighbourhood_radius: Maximum distance allowed to travel for each person from his initial location
    :param epochs:               Steps to perform during each people 1) travel and 2) spread the disease
    :param radius:               Maximum radius for person to travel in single epoch
    :param spread_radius:        Disease spreading radius
    :param infect_prob:          Base probability for disease to transmit
    :param plot_disease_matrix:  Path to save plot of disease (exposure) matrix before transmitting a disease in each epoch
    :return:                     Number of ill (visible + invisible) people for each epoch, number of ill (visible) people that can transmit a disease for each epoch
    """

    ###########################
    # Init variables          #
    ###########################

    # Init people arrays
    people_x_arr = np.zeros(people_num).astype(int)
    people_y_arr = np.zeros(people_num).astype(int)
    people_status_arr = np.zeros(people_num).astype(int)  # Ill or not (True)
    people_timer_arr = np.zeros(people_num).astype(int)  # Timer untill illnes becomes observable
    people_observ_arr = np.zeros(people_num).astype(int)  # Illness or not (observable only)

    # Randomly initiate infected group
    assert people_num == people_status_arr.size
    people_status_arr[np.random.choice(people_num, infected_init_num, replace=False)] = 1
    people_observ_arr = people_status_arr.copy()

    # Init a grid (matrix)
    mat = np.zeros(shape=(y_size, x_size)).astype(int)

    ###########################
    # Init matrix with people #
    ###########################

    # Loop through each person to place him on a grid
    for i in range(people_status_arr.size):

        # Make that matrix is square
        assert x_size == y_size
        x, y = np.random.randint(x_size, size=2)
        people_x_arr[i], people_y_arr[i] = x, y
        mat[y, x] += 1

    init_people_x_arr = people_x_arr.copy()
    init_people_y_arr = people_y_arr.copy()
    
    ###########################
    # Run simulations         #
    ###########################

    disease_tracker = []
    visible_disease_tracker = []
    
    for i in tqdm(range(epochs)):

        # Make disease visible (and transmittable)
        if i > 0:
            
            # Make illness observable
            new_disease_observations = np.where(people_timer_arr == 1)[0]
            people_observ_arr[new_disease_observations] = 1

            # Decrement people counter with unabservable illness
            disease_indices = np.where(people_timer_arr > 0)[0]
            people_timer_arr[disease_indices] -= 1
        
        # Transport people
        transport_iter(people_x_arr, people_y_arr, init_people_x_arr, init_people_y_arr, x_size, y_size, radius)

        # Observe disease map
        disease_mat = make_disease_matrix(people_x_arr, people_y_arr, people_observ_arr, x_size, y_size, spread_radius)
        
        # Plot & save disease exposure map
        if plot_disease_matrix is not None:
            plot_disease_exposure(people_x_arr, people_y_arr, people_observ_arr, people_status_arr, spread_radius, epoch=i, path=plot_disease_matrix)

        # Spread disease (based on the map above)
        spread_disease(disease_mat, people_x_arr, people_y_arr, people_status_arr, people_timer_arr, infect_prob)
        
        # Track stats
        disease_tracker.append(people_status_arr.sum())
        visible_disease_tracker.append(people_observ_arr.sum())
        
        # Debug
        print('[epoch={}]\tinfected(total)={}\ttransmitters(visibly ill)={}'.format(i, people_status_arr.sum(), people_observ_arr.sum()))
        
    return np.array(disease_tracker), np.array(visible_disease_tracker)

In [ ]:
people_num = 30000           # Number of people in simulation
x_size, y_size = 3000, 3000  # Grid sizes (map to place people into)
infected_init_num = 100      # Initial infected people number
timer_start = 12             # Steps (epochs) until infected person can transmit a disease (exception: initial group)
neighbourhood_radius = 120   # Maximum distance allowed to travel for each person from his initial location
epochs = 120                 # Steps to perform during each people 1) travel and 2) spread the disease

radius = 1.5                 # Maximum radius for person to travel in single epoch
spread_radius = 20           # Disease spreading radius
infect_prob = 1.0            # Base probability for disease to transmit

# Path to plot of disease (exposure) matrix before transmitting a disease in each epoch
plot_disease_matrix = r'C:\Users\va\Transport_networks\radius_{}_spread_radius_{}_infected_prob_{}'.format(str(radius).replace('.', '_'),
                                                                                                              str(spread_radius).replace('.', '_'),
                                                                                                              str(infect_prob).replace('.', '_'))

# Function to perform simulation
disease_tracker, visible_disease_tracker = simulate_transportations_with_infections(people_num, \
                                                                                    x_size, y_size, \
                                                                                    infected_init_num, \
                                                                                    timer_start, \
                                                                                    neighbourhood_radius, \
                                                                                    infect_prob, \
                                                                                    radius, \
                                                                                    spread_radius, \
                                                                                    epochs, \
                                                                                    plot_disease_matrix)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(disease_tracker, '.-', c='tab:blue', label='Фдд infected people')
ax.plot(visible_disease_tracker, '.-', c='tab:red', label='Visible (transmitable) infected people')
ax.set_ylabel('Infected')
ax.set_xlabel('Time')
ax.set_title(f'Infected vs. time (radius={radius}, spread_radius={spread_radius}, infect_prob={infect_prob})')
ax.grid()
ax.legend()

fig.savefig(os.path.join(f'Transport_networks_{x_size}x{y_size}_{people_num}_people_{epochs}_epochs_infected_ts.png'), dpi=300);

plt.show()